In [2]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

artists_data_path = "../data/artists.jsonl"
sessions_data_path = "../data/sessions.jsonl"
track_storage_data_path = "../data/track_storage.jsonl"
tracks_data_path = "../data/tracks.jsonl"
users_data_path = "../data/users.jsonl"

artists_data_frame = pd.read_json(artists_data_path, lines=True)
sessions_data_frame = pd.read_json(sessions_data_path, lines=True)
tracks_data_frame = pd.read_json(tracks_data_path, lines=True)
users_data_frame = pd.read_json(users_data_path, lines=True)

# Mergowanie podobnych gatunków muzycznych (artists)
TODO


# Scalenie danych

In [3]:
merged_data_frame = pd.merge(sessions_data_frame, tracks_data_frame, left_on="track_id",
                             right_on="id")
merged_data_frame = pd.merge(merged_data_frame, users_data_frame, left_on="user_id",
                             right_on="user_id")
merged_data_frame = pd.merge(merged_data_frame, artists_data_frame, left_on="id_artist",
                             right_on="id")
merged_data_frame.head(5)


,timestamp,user_id,track_id,event_type,session_id,id_x,name_x,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,name_y,city,street,favourite_genres,premium_user,id_y,name,genres
0,2023-01-03 05:09:55.000,101,2PmGtDUyJIpYBEtI1hQIVp,PLAY,124,2PmGtDUyJIpYBEtI1hQIVp,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,Klara Herdzik,Szczecin,pl. Floriana 55/22,"[permanent wave, mandopop, funk]",False,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"
1,2023-02-08 21:41:02.507,103,2PmGtDUyJIpYBEtI1hQIVp,PLAY,151,2PmGtDUyJIpYBEtI1hQIVp,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,Szymon Zawal,Gdynia,al. Głogowa 14/10,"[filmi, regional mexican, folk]",False,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"
2,2023-02-12 01:17:14.946,132,2PmGtDUyJIpYBEtI1hQIVp,PLAY,544,2PmGtDUyJIpYBEtI1hQIVp,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"
3,2023-01-03 03:59:59.738,132,5yxYokipsWlpDCt4Th4VVc,PLAY,534,5yxYokipsWlpDCt4Th4VVc,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"
4,2023-03-07 14:28:25.702,132,5yxYokipsWlpDCt4Th4VVc,PLAY,547,5yxYokipsWlpDCt4Th4VVc,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"


## Wyliczenie, czy dana piosenka w danej sesji została pominięta

In [4]:
# delete rows with event_type other than PLAY and SKIP
merged_data_frame = merged_data_frame[merged_data_frame["event_type"].isin(["PLAY", "SKIP"])]

# TODO moze to trzeba liczyc inaczej?

# if for one user_id and track_id and session_id there is only PLAY event, then the track was not skipped
merged_data_frame["skipped"] = merged_data_frame.groupby(["user_id", "track_id", "session_id"])[
  "event_type"].transform(lambda x: x != "PLAY").astype(bool)

# if there are 2 rows with the same user_id, track_id and session_id, and different "skipped" value, remove the row with "skipped" = 0
merged_data_frame = merged_data_frame.drop_duplicates(subset=["user_id", "track_id", "session_id"],
                                                      keep="last")

## Usunięcie niepotrzebnych kolumn

In [5]:
merged_data_frame = merged_data_frame.drop(columns=["id_x", "id_y", "name", "premium_user", "name_x", "name_y", "city", "street", "id_artist", "event_type"])
merged_data_frame.head(10)

# save to jsonl file
merged_data_frame.to_json("../data/merged_data.jsonl", orient="records", lines=True)
